In [1]:
import pandas as pd
import numpy as np


In [2]:
airport_path = "/Users/xiaoxiaoren/Documents/sjsu s2 class/257 Machine Learning/project/groupProject/dataset/flight-delays/airports.csv"
airline_path = "/Users/xiaoxiaoren/Documents/sjsu s2 class/257 Machine Learning/project/groupProject/dataset/flight-delays/airlines.csv"
flights_path  ="/Users/xiaoxiaoren/Documents/sjsu s2 class/257 Machine Learning/project/groupProject/dataset/flight-delays/flights.csv"

In [3]:
airports = pd.read_csv(airport_path)

In [4]:
airlines = pd.read_csv(airline_path)

In [5]:
flights = pd.read_csv(flights_path)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
len(flights.columns)

31

In [7]:
flights_sfo = flights.drop([ 'DIVERTED','TAXI_OUT',
 'WHEELS_OFF','WHEELS_ON',
 'TAXI_IN',
 'SCHEDULED_ARRIVAL',
 'ARRIVAL_TIME',
 'ARRIVAL_DELAY','CANCELLED',
 'CANCELLATION_REASON',
 'AIR_SYSTEM_DELAY',
 'SECURITY_DELAY',
 'AIRLINE_DELAY',
 'LATE_AIRCRAFT_DELAY',
 'WEATHER_DELAY',
'SCHEDULED_TIME',
 'ELAPSED_TIME',
 'AIR_TIME',
 'DISTANCE',
    'DESTINATION_AIRPORT',
    'DEPARTURE_TIME','TAIL_NUMBER',
                           ], axis=1)

In [8]:
flights_sfo.drop(['FLIGHT_NUMBER'], axis=1, inplace=True)

In [9]:
flights_sfo.columns

Index(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'ORIGIN_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_DELAY'],
      dtype='object')

In [10]:
flights_sfo = flights_sfo.loc[flights['ORIGIN_AIRPORT'] == 'SFO']
flights_sfo.head(10)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_DELAY
2,2015,1,1,4,US,SFO,20,-2.0
5,2015,1,1,4,DL,SFO,25,-5.0
8,2015,1,1,4,AA,SFO,30,-11.0
17,2015,1,1,4,UA,SFO,48,-6.0
92,2015,1,1,4,UA,SFO,531,19.0
155,2015,1,1,4,WN,SFO,555,-2.0
190,2015,1,1,4,WN,SFO,600,-6.0
191,2015,1,1,4,UA,SFO,600,-3.0
195,2015,1,1,4,UA,SFO,600,5.0
202,2015,1,1,4,US,SFO,600,-3.0


In [11]:
flights_sfo['DEPARTURE_DELAY'] = np.where(flights_sfo['DEPARTURE_DELAY']>0, 1, 0)
flights_sfo.tail(10)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_DELAY
5818966,2015,12,31,4,UA,SFO,2304,0
5818967,2015,12,31,4,UA,SFO,2304,1
5818971,2015,12,31,4,UA,SFO,2308,1
5818973,2015,12,31,4,UA,SFO,2309,0
5818981,2015,12,31,4,UA,SFO,2313,0
5819000,2015,12,31,4,OO,SFO,2325,0
5819005,2015,12,31,4,OO,SFO,2330,0
5819007,2015,12,31,4,UA,SFO,2330,0
5819024,2015,12,31,4,UA,SFO,2343,0
5819041,2015,12,31,4,UA,SFO,2354,0


In [23]:
dummy = pd.get_dummies(flights_sfo.DAY_OF_WEEK)
dummy.drop(dummy.columns[len(dummy.columns)-1], axis=1, inplace=True)
dummy.columns = ['Mon', 'Tue','Wed','Thu','Fri','Sat']
flights_sfo = pd.concat([flights_sfo,dummy],axis=1)
flights_sfo.drop(['DAY_OF_WEEK','ORIGIN_AIRPORT'], axis=1, inplace=True)

In [24]:
flights_sfo.SCHEDULED_DEPARTURE=flights_sfo.SCHEDULED_DEPARTURE.apply(lambda x: str(x).zfill(4))
flights_sfo.SCHEDULED_DEPARTURE = flights_sfo.SCHEDULED_DEPARTURE.str[:2] + ':' + flights_sfo.SCHEDULED_DEPARTURE.str[-2:]
flights_sfo_datetime = pd.to_datetime(dict(year=flights_sfo.YEAR, month=flights_sfo.MONTH, day=flights_sfo.DAY,hour =flights_sfo.SCHEDULED_DEPARTURE.str[:2],minute =flights_sfo.SCHEDULED_DEPARTURE.str[3:] ))
flights_sfo = pd.concat([flights_sfo,flights_sfo_datetime],axis=1)
flights_sfo.drop(['YEAR','MONTH','DAY','SCHEDULED_DEPARTURE'], axis=1, inplace=True)


In [25]:
flights_sfo.rename(columns={flights_sfo.columns[-1]: 'Timestamp'}, inplace=True)
flights_sfo.head()

,AIRLINE,DEPARTURE_DELAY,Mon,Tue,Wed,Thu,Fri,Sat,Timestamp
2,US,0,0,0,0,1,0,0,2015-01-01 00:20:00
5,DL,0,0,0,0,1,0,0,2015-01-01 00:25:00
8,AA,0,0,0,0,1,0,0,2015-01-01 00:30:00
17,UA,0,0,0,0,1,0,0,2015-01-01 00:48:00
92,UA,1,0,0,0,1,0,0,2015-01-01 05:31:00


In [72]:
flights_sfo.Timestamp.tail(10)

5818966   2015-12-31 23:04:00
5818967   2015-12-31 23:04:00
5818971   2015-12-31 23:08:00
5818973   2015-12-31 23:09:00
5818981   2015-12-31 23:13:00
5819000   2015-12-31 23:25:00
5819005   2015-12-31 23:30:00
5819007   2015-12-31 23:30:00
5819024   2015-12-31 23:43:00
5819041   2015-12-31 23:54:00
Name: Timestamp, dtype: datetime64[ns]

In [116]:
sfo = pd.read_csv("/Users/xiaoxiaoren/Documents/sjsu s2 class/257 Machine Learning/project/groupProject/dataset/SFO.csv", encoding="ascii")

In [117]:
sfo_dropList = ['station', 'skyc1', 'skyc2', 'skyc3', 'skyc4', 'skyl1', 'skyl2', 'skyl3', 'skyl4', 'wxcodes', 'metar']
sfo.drop(sfo_dropList, axis=1, inplace=True)
sfo.head(10)

,valid,tmpf,dwpf,relh,drct,sknt,p01i,alti,mslp,vsby,gust
0,2015-01-01 00:56,53.06,32,44.48,280,7,0.0,30.09,1018.9,10,M
1,2015-01-01 01:56,51.98,33.08,48.34,270,6,0.0,30.08,1018.7,10,M
2,2015-01-01 02:56,50,37.04,60.89,280,8,0.0,30.1,1019.1,10,M
3,2015-01-01 03:56,50,37.04,60.89,270,5,0.0,30.09,1018.7,10,M
4,2015-01-01 04:56,50,37.04,60.89,0,0,0.0,30.08,1018.5,10,M
5,2015-01-01 05:56,44.96,35.96,70.52,150,4,0.0,30.08,1018.7,10,M
6,2015-01-01 06:56,44.96,28.94,53.17,0,0,0.0,30.08,1018.6,10,M
7,2015-01-01 07:56,42.98,30.92,62.16,210,4,0.0,30.08,1018.6,10,M
8,2015-01-01 08:56,44.06,33.08,65.07,300,3,0.0,30.08,1018.4,10,M
9,2015-01-01 09:56,39.92,32,73.08,200,3,0.0,30.09,1018.8,10,M


In [119]:
sfo['valid']= pd.to_datetime(sfo['valid'], format='%Y%m%d %H:%M')
sfo.head(10)

,valid,tmpf,dwpf,relh,drct,sknt,p01i,alti,mslp,vsby,gust
0,2015-01-01 00:56:00,53.06,32,44.48,280,7,0.0,30.09,1018.9,10,M
1,2015-01-01 01:56:00,51.98,33.08,48.34,270,6,0.0,30.08,1018.7,10,M
2,2015-01-01 02:56:00,50,37.04,60.89,280,8,0.0,30.1,1019.1,10,M
3,2015-01-01 03:56:00,50,37.04,60.89,270,5,0.0,30.09,1018.7,10,M
4,2015-01-01 04:56:00,50,37.04,60.89,0,0,0.0,30.08,1018.5,10,M
5,2015-01-01 05:56:00,44.96,35.96,70.52,150,4,0.0,30.08,1018.7,10,M
6,2015-01-01 06:56:00,44.96,28.94,53.17,0,0,0.0,30.08,1018.6,10,M
7,2015-01-01 07:56:00,42.98,30.92,62.16,210,4,0.0,30.08,1018.6,10,M
8,2015-01-01 08:56:00,44.06,33.08,65.07,300,3,0.0,30.08,1018.4,10,M
9,2015-01-01 09:56:00,39.92,32,73.08,200,3,0.0,30.09,1018.8,10,M


In [125]:
sfo.rename(columns={"valid": "Time", "tmpf": "Air_Temperature", "dwpf": "Dew_Point_Temperature", 
                    "relh": "Relative_Humidity", "drct": "Wind_Direction", "sknt": "Wind_Speed",
                    "p01i": "Precipitation_Per_Hour", "alti": "Pressure_Altimeter", "mslp": "Sea_Level_Pressure",
                    "vsby": "Visibility", "gust": "Wind_Gust"}, inplace = True)

In [174]:
TimestampYearToHour = pd.Series(flights_sfo.Timestamp)

for i in TimestampYearToHour.index:
    TimestampYearToHour[i] = TimestampYearToHour[i].strftime("%H, %B %d, %Y")
    
print(TimestampYearToHour)

2           00, January 01, 2015
5           00, January 01, 2015
8           00, January 01, 2015
17          00, January 01, 2015
92          05, January 01, 2015
155         05, January 01, 2015
190         06, January 01, 2015
191         06, January 01, 2015
195         06, January 01, 2015
202         06, January 01, 2015
214         06, January 01, 2015
302         06, January 01, 2015
363         06, January 01, 2015
395         06, January 01, 2015
401         06, January 01, 2015
416         06, January 01, 2015
482         06, January 01, 2015
588         06, January 01, 2015
607         06, January 01, 2015
630         06, January 01, 2015
687         06, January 01, 2015
707         06, January 01, 2015
809         06, January 01, 2015
843         07, January 01, 2015
845         07, January 01, 2015
870         07, January 01, 2015
892         07, January 01, 2015
918         07, January 01, 2015
983         07, January 01, 2015
985         07, January 01, 2015
          

In [176]:
flights_sfo['TimestampYearToHour'] = TimestampYearToHour
flights_sfo.head(10)

,AIRLINE,DEPARTURE_DELAY,Mon,Tue,Wed,Thu,Fri,Sat,Timestamp,TimestampYearToHour
2,US,0,0,0,0,1,0,0,2015-01-01 00:20:00,"00, January 01, 2015"
5,DL,0,0,0,0,1,0,0,2015-01-01 00:25:00,"00, January 01, 2015"
8,AA,0,0,0,0,1,0,0,2015-01-01 00:30:00,"00, January 01, 2015"
17,UA,0,0,0,0,1,0,0,2015-01-01 00:48:00,"00, January 01, 2015"
92,UA,1,0,0,0,1,0,0,2015-01-01 05:31:00,"05, January 01, 2015"
155,WN,0,0,0,0,1,0,0,2015-01-01 05:55:00,"05, January 01, 2015"
190,WN,0,0,0,0,1,0,0,2015-01-01 06:00:00,"06, January 01, 2015"
191,UA,0,0,0,0,1,0,0,2015-01-01 06:00:00,"06, January 01, 2015"
195,UA,1,0,0,0,1,0,0,2015-01-01 06:00:00,"06, January 01, 2015"
202,US,0,0,0,0,1,0,0,2015-01-01 06:00:00,"06, January 01, 2015"


In [177]:
TimestampYearToHour = pd.Series(sfo.Time)

for i in TimestampYearToHour.index:
    TimestampYearToHour[i] = TimestampYearToHour[i].strftime("%H, %B %d, %Y")
    
print(TimestampYearToHour)

0        00, January 01, 2015
1        01, January 01, 2015
2        02, January 01, 2015
3        03, January 01, 2015
4        04, January 01, 2015
5        05, January 01, 2015
6        06, January 01, 2015
7        07, January 01, 2015
8        08, January 01, 2015
9        09, January 01, 2015
10       10, January 01, 2015
11       11, January 01, 2015
12       12, January 01, 2015
13       13, January 01, 2015
14       14, January 01, 2015
15       15, January 01, 2015
16       16, January 01, 2015
17       17, January 01, 2015
18       18, January 01, 2015
19       19, January 01, 2015
20       20, January 01, 2015
21       21, January 01, 2015
22       22, January 01, 2015
23       23, January 01, 2015
24       00, January 02, 2015
25       01, January 02, 2015
26       02, January 02, 2015
27       03, January 02, 2015
28       04, January 02, 2015
29       05, January 02, 2015
                ...          
9501    21, December 29, 2015
9502    22, December 29, 2015
9503    23

In [178]:
sfo['TimestampYearToHour'] = TimestampYearToHour
sfo.head(10)

,Time,AirTemperature,DewPointTemperature,RelativeHumidity,WindDirection,WindSpeed,PrecipitationPerHour,PressureAltimeter,SeaLevelPressure,Visibility,WindGust,TimestampYearToHour
0,2015-01-01 00:56:00,53.06,32,44.48,280,7,0.0,30.09,1018.9,10,M,"00, January 01, 2015"
1,2015-01-01 01:56:00,51.98,33.08,48.34,270,6,0.0,30.08,1018.7,10,M,"01, January 01, 2015"
2,2015-01-01 02:56:00,50,37.04,60.89,280,8,0.0,30.1,1019.1,10,M,"02, January 01, 2015"
3,2015-01-01 03:56:00,50,37.04,60.89,270,5,0.0,30.09,1018.7,10,M,"03, January 01, 2015"
4,2015-01-01 04:56:00,50,37.04,60.89,0,0,0.0,30.08,1018.5,10,M,"04, January 01, 2015"
5,2015-01-01 05:56:00,44.96,35.96,70.52,150,4,0.0,30.08,1018.7,10,M,"05, January 01, 2015"
6,2015-01-01 06:56:00,44.96,28.94,53.17,0,0,0.0,30.08,1018.6,10,M,"06, January 01, 2015"
7,2015-01-01 07:56:00,42.98,30.92,62.16,210,4,0.0,30.08,1018.6,10,M,"07, January 01, 2015"
8,2015-01-01 08:56:00,44.06,33.08,65.07,300,3,0.0,30.08,1018.4,10,M,"08, January 01, 2015"
9,2015-01-01 09:56:00,39.92,32,73.08,200,3,0.0,30.09,1018.8,10,M,"09, January 01, 2015"


In [190]:
sfo = sfo.sample(frac=1).drop_duplicates(subset='TimestampYearToHour')
sfo

,Time,AirTemperature,DewPointTemperature,RelativeHumidity,WindDirection,WindSpeed,PrecipitationPerHour,PressureAltimeter,SeaLevelPressure,Visibility,WindGust,TimestampYearToHour
7743,2015-10-24 21:56:00,64.94,55.94,72.62,290,20,0.00,29.95,1014.2,10,M,"21, October 24, 2015"
5986,2015-08-16 20:56:00,80.06,64.94,59.99,360,6,0.00,29.86,1011.1,10,M,"20, August 16, 2015"
5863,2015-08-12 04:56:00,62.06,55.94,80.34,260,9,0.00,29.98,1015.1,10,M,"04, August 12, 2015"
1183,2015-02-14 17:56:00,60.98,57.02,86.79,0,0,0.00,30.12,1019.9,10,M,"17, February 14, 2015"
169,2015-01-08 00:56:00,57.02,48.02,71.81,20,4,0.00,30.1,1019.2,10,M,"00, January 08, 2015"
6700,2015-09-13 22:56:00,66.02,55.94,69.94,300,16,0.00,29.76,1007.6,10,M,"22, September 13, 2015"
4175,2015-06-07 22:56:00,66.92,53.96,63.08,300,21,0.00,29.85,1010.6,10,27,"22, June 07, 2015"
5746,2015-08-07 23:56:00,73.94,55.94,53.37,210,11,0.00,29.86,1011.2,10,M,"23, August 07, 2015"
9471,2015-12-28 15:56:00,41,35.06,79.21,30,7,0.00,30.09,1019,10,M,"15, December 28, 2015"
5307,2015-07-21 19:56:00,69.08,57.02,65.44,270,13,0.00,29.91,1012.8,10,18,"19, July 21, 2015"


In [191]:
mergedTable = pd.merge(flights_sfo, sfo, how='inner', on='TimestampYearToHour')
mergedTable[['Time','Timestamp', 'TimestampYearToHour']]

,Time,Timestamp,TimestampYearToHour
0,2015-01-01 00:56:00,2015-01-01 00:20:00,"00, January 01, 2015"
1,2015-01-01 00:56:00,2015-01-01 00:25:00,"00, January 01, 2015"
2,2015-01-01 00:56:00,2015-01-01 00:30:00,"00, January 01, 2015"
3,2015-01-01 00:56:00,2015-01-01 00:48:00,"00, January 01, 2015"
4,2015-01-01 05:56:00,2015-01-01 05:31:00,"05, January 01, 2015"
5,2015-01-01 05:56:00,2015-01-01 05:55:00,"05, January 01, 2015"
6,2015-01-01 06:56:00,2015-01-01 06:00:00,"06, January 01, 2015"
7,2015-01-01 06:56:00,2015-01-01 06:00:00,"06, January 01, 2015"
8,2015-01-01 06:56:00,2015-01-01 06:00:00,"06, January 01, 2015"
9,2015-01-01 06:56:00,2015-01-01 06:00:00,"06, January 01, 2015"


In [192]:
mergedTable.drop(['Time', 'TimestampYearToHour'], axis=1, inplace=True)

In [193]:
mergedTable.head(10)

,AIRLINE,DEPARTURE_DELAY,Mon,Tue,Wed,Thu,Fri,Sat,Timestamp,AirTemperature,DewPointTemperature,RelativeHumidity,WindDirection,WindSpeed,PrecipitationPerHour,PressureAltimeter,SeaLevelPressure,Visibility,WindGust
0,US,0,0,0,0,1,0,0,2015-01-01 00:20:00,53.06,32,44.48,280,7,0.0,30.09,1018.9,10,M
1,DL,0,0,0,0,1,0,0,2015-01-01 00:25:00,53.06,32,44.48,280,7,0.0,30.09,1018.9,10,M
2,AA,0,0,0,0,1,0,0,2015-01-01 00:30:00,53.06,32,44.48,280,7,0.0,30.09,1018.9,10,M
3,UA,0,0,0,0,1,0,0,2015-01-01 00:48:00,53.06,32,44.48,280,7,0.0,30.09,1018.9,10,M
4,UA,1,0,0,0,1,0,0,2015-01-01 05:31:00,44.96,35.96,70.52,150,4,0.0,30.08,1018.7,10,M
5,WN,0,0,0,0,1,0,0,2015-01-01 05:55:00,44.96,35.96,70.52,150,4,0.0,30.08,1018.7,10,M
6,WN,0,0,0,0,1,0,0,2015-01-01 06:00:00,44.96,28.94,53.17,0,0,0.0,30.08,1018.6,10,M
7,UA,0,0,0,0,1,0,0,2015-01-01 06:00:00,44.96,28.94,53.17,0,0,0.0,30.08,1018.6,10,M
8,UA,1,0,0,0,1,0,0,2015-01-01 06:00:00,44.96,28.94,53.17,0,0,0.0,30.08,1018.6,10,M
9,US,0,0,0,0,1,0,0,2015-01-01 06:00:00,44.96,28.94,53.17,0,0,0.0,30.08,1018.6,10,M


In [195]:
missing_df = mergedTable.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(sfo.shape[0]-missing_df['missing values'])/sfo.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)

,variable,missing values,filling factor (%)
0,AIRLINE,0,100.0
1,SeaLevelPressure,0,100.0
2,PressureAltimeter,0,100.0
3,PrecipitationPerHour,0,100.0
4,WindSpeed,0,100.0
5,WindDirection,0,100.0
6,RelativeHumidity,0,100.0
7,DewPointTemperature,0,100.0
8,Visibility,0,100.0
9,AirTemperature,0,100.0
